<a href="https://colab.research.google.com/github/Vietanh-Vu/Machine-learning-algorithm/blob/main/Detect_Comment_Social_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade git+https://github.com/jpmml/sklearn2pmml.git

  Cloning https://github.com/jpmml/sklearn2pmml.git to /tmp/pip-req-build-y_9ol0y4
  Running command git clone --filter=blob:none --quiet https://github.com/jpmml/sklearn2pmml.git /tmp/pip-req-build-y_9ol0y4
  Resolved https://github.com/jpmml/sklearn2pmml.git to commit 3ab1489f7cd421cd57652c4f83743f4ed9ba211e
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import joblib
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Comment/train-comment.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26799 entries, 0 to 26798
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   free_text  26797 non-null  object
 1   label_id   26799 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 418.9+ KB


In [ ]:
validate = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Comment/validate-comment.csv')
validate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   free_text  2672 non-null   object
 1   label_id   2672 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 41.9+ KB


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Comment/test-comment.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6680 entries, 0 to 6679
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   free_text  6680 non-null   object
 1   label_id   6680 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 104.5+ KB


In [ ]:
train['free_text'].fillna('', inplace=True)
validate['free_text'].fillna('', inplace=True)
test['free_text'].fillna('', inplace=True)

In [ ]:
X_train = train['free_text']
y_train = train['label_id']
X_test = test['free_text']
y_test = test['label_id']

In [ ]:
pipeline = PMMLPipeline([
    ("vectorizer", CountVectorizer()),
    ('classifier', LogisticRegression())
])

In [ ]:
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


PMMLPipeline(steps=[('vectorizer', CountVectorizer()),
       ('classifier', LogisticRegression())])

In [ ]:
# Make predictions
y_pred = pipeline.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8907185628742516


In [ ]:
# Save the model to PMML
sklearn2pmml(pipeline, "/content/drive/MyDrive/ColabNotebooks/Comment/comment-model.pmml", with_repr=True)

In [ ]:
!pip install pypmml
from pypmml import Model

In [ ]:

model = Model.load("/content/drive/MyDrive/ColabNotebooks/Comment/comment-model.pmml")
def predict_comment(comment):
    # Prepare the input
    input_df = pd.DataFrame({'free_text': [comment]})

    # Make prediction
    prediction = model.predict(input_df)
    return prediction
    print(f"% Not spam: {prediction['probability(0)'][0]}")
    print(f"% Spam: {prediction['probability(1)'][0]}")


# Test the model with a sample comment
sample_comment = "dit"
prediction = predict_comment(sample_comment)
prediction

,probability(0),probability(1)
0,0.41025,0.58975
